In [ ]:
# Instalar o pacotes quando necessário
pip install tiktoken

In [1]:
# Carregar os pacotes necessários
import os
import openai
import pypdf
import pandas as pd
from dotenv import load_dotenv
from dotenv import dotenv_values
import certifi
import ssl
import tiktoken

In [8]:
# Carregar certificado SSL
ssl._create_default_https_context = ssl.create_default_context(cafile=certifi.where())

In [2]:
# Carregar variáveis de ambiente
load_dotenv()

# Obter a chave da API
openai.api_key = os.getenv("OPENAI_API_KEY")

# Verificar se a chave foi carregada corretamente
if not openai.api_key:
    raise ValueError("A chave da API não foi carregada. Verifique o arquivo .env")

In [3]:
# Função que extrai texto de um PDF
def extract_text_from_pdf(pdf_path):
    """Extrai texto de um PDF."""
    reader = pypdf.PdfReader(pdf_path)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text.strip()

In [4]:
# Função que envia o texto para a OpenAI e retorna a análise
def analyze_text_with_openai(text):
    """Envia o texto para a OpenAI e retorna a análise."""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Extraia as informações mais relevantes do texto."},
            {"role": "user", "content": text}
        ]
    )
    return response["choices"][0]["message"]["content"]

In [5]:
# Função que salva a saída em um arquivo TXT
def save_output_as_txt(output_text, output_file="output.txt"):
    """Salva a saída em um arquivo TXT."""
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(output_text)
    print(f"✅ Resultado salvo em {output_file}")

# Função que salva a saída em um arquivo CSV
def save_output_as_csv(output_text, output_file="output.csv"):
    """Salva a saída em um arquivo CSV."""
    df = pd.DataFrame({"Informações Extraídas": [output_text]})
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"✅ Resultado salvo em {output_file}")

In [6]:
# Caminho do arquivo PDF
pdf_path = "/Users/anderson.paiva/Downloads/exames.pdf"  # Altere para o caminho do seu PDF

if not os.path.exists(pdf_path):
    print(f"Arquivo {pdf_path} não encontrado!")
else:
    print("📖 Lendo o PDF...")
    text = extract_text_from_pdf(pdf_path)
    
    if text:
        print("🚀 Enviando para OpenAI...")
        extracted_info = analyze_text_with_openai(text)

        # Salvar os resultados
        save_output_as_txt(extracted_info)
        save_output_as_csv(extracted_info)

        # Exibir saída no Notebook
        print("\n📌 Informações Extraídas:")
        print(extracted_info)
    else:
        print("❌ Não foi possível extrair texto do PDF.")

📖 Lendo o PDF...
🚀 Enviando para OpenAI...
✅ Resultado salvo em output.txt
✅ Resultado salvo em output.csv

📌 Informações Extraídas:
- O documento apresenta um pedido de exames médicos realizado pelo paciente Anderson Xavier de Paiva Mello, assinado digitalmente pela Dra. Rosiane Aparecida Turim Gomes Pinho, com CRM 185007 SP.
- Dentre os exames solicitados, destacam-se hemograma completo, colesterol total, glicemia em jejum, hormônio tireoestimulante (TSH), creatinina, entre outros. Além disso, inclui exames como Proteína C reativa, VDRL, HBsAg, Anti-HIV 1 e 2, Anti-HCV, entre outros.
- Também foi solicitada uma radiografia de tórax com 2 incidências, além de um exame de rotina de urina com urocultura.
- O acesso à receita digital via QR Code e a validação da assinatura do documento podem ser feitas por meio do link https://validador.memed.com.br utilizando o Token: RfpgQt.
- O paciente possui o CPF 314.596.618-93 e o endereço fornecido para o exame é Rua Dona Adma Jafet 91.
- O códig

Número de tokens no arquivo: 277


In [10]:
# Dicionário com preços dos modelos
precos_modelos = {
    "gpt-4-turbo": (0.01, 0.03),
    "gpt-4": (0.03, 0.06),
    "gpt-3.5-turbo": (0.0015, 0.002),
    "text-davinci-003": (0.02, 0.02),
    "code-davinci-002": (0.02, 0.02),
}

# Função que conta o número de tokens de um arquivo
def contar_tokens_arquivo(caminho_arquivo, modelo="gpt-3.5-turbo"):

    encoding = tiktoken.encoding_for_model(modelo)

    with open(caminho_arquivo, "r", encoding="utf-8") as f:
        texto = f.read()

    return len(encoding.encode(texto))

# Função que calcula o custo de um arquivo
def calcular_custo(tokens_saida):
    """ Calcula o custo baseado no número de tokens de saída para todos os modelos. """
    custos = {}
    for modelo, (preco_entrada, preco_saida) in precos_modelos.items():
        custo_total = (tokens_saida / 1000) * preco_saida  # Apenas saída, assumindo que já está gerado
        custos[modelo] = round(custo_total, 6)  # Arredondando para 6 casas decimais
    
    return custos

# Caminho do arquivo de saída
caminho_arquivo = "/Users/anderson.paiva/Documents/GitHub/axpm_projetos_dados/projeto_pdf_extract/output.txt"

# Contar tokens do arquivo
tokens_saida = contar_tokens_arquivo(caminho_arquivo)

# Calcular custo para cada modelo
custos = calcular_custo(tokens_saida)

# Exibir os custos
print(f"Número de tokens no arquivo: {tokens_saida}")
for modelo, custo in custos.items():
    print(f"Custo para {modelo}: ${custo}")

Número de tokens no arquivo: 277
Custo para gpt-4-turbo: $0.00831
Custo para gpt-4: $0.01662
Custo para gpt-3.5-turbo: $0.000554
Custo para text-davinci-003: $0.00554
Custo para code-davinci-002: $0.00554
